# DataFrame com Dados dos Filmes (movies)

In [2]:
colunas_info_movie = ['id_hashed', 'movie_title', 'color', 'duration', 'genres', 'plot_keywords', 'language', 'country', 'content_rating', 'title_year', 'aspect_ratio','movie_imdb_link']

In [3]:
df.select(colunas_info_movie).write.format('parquet')\
.option("header","true")\
.mode('overwrite')\
.save("dbfs:/FileStore/trusted/info_movie")

NameError: name 'df' is not defined

In [ ]:
df_info_movie = spark.read.parquet("dbfs:/FileStore/trusted/info_movie")
df_info_movie.createOrReplaceTempView('df_info_movie')
display(df_info_movie)

### Analise Matheus

In [ ]:
%sql
-- selecionando filmes do gênero 
select movie_title, genres from df_info_movie
where genres like '%Comedy%'

In [ ]:
%sql
-- selecionando filmes de 2015 para frente
select movie_title, title_year from df_info_movie
where title_year > 2015

In [ ]:
%sql
-- selecionando o diretor do filme
select director_name, movie_title, df_cast.id_hashed from df_cast
inner join df_info_movie
on df_cast.id_hashed = df_info_movie.id_hashed

In [ ]:
#Frequencia de palavras chaves
import numpy as np
dado = df_info_movie.select('plot_keywords')

lista = []
dado_junto = dado.collect()

for i in range(len(dado_junto)):
    if dado_junto[i][0] != None:
        separ = dado_junto[i][0].split('|')
        for i in separ:
            lista.append(i)

freq = {}

for item in lista:
    if item in freq:
        freq[item] += 1
    else:
        freq[item] = 1
        
frequencia_keywords = spark.createDataFrame(data=freq.items(), schema=['keywords', 'frequency'])
frequencia_keywords.createOrReplaceTempView('frequencia_keywords')
display(frequencia_keywords)

In [ ]:
%sql
SELECT * from frequencia_keywords
ORDER BY frequency DESC

In [ ]:
#Frequencia de genero
import numpy as np
dado_genero = df_info_movie.select('genres')

lista_genero = []
dado_junto_genero = dado_genero.collect()

for i in range(len(dado_junto_genero)):
    if dado_junto_genero[i][0] != None:
        separ = dado_junto_genero[i][0].split('|')
        for i in separ:
            lista_genero.append(i)

freq_genero = {}

for item in lista_genero:
    if item in freq_genero:
        freq_genero[item] += 1
    else:
        freq_genero[item] = 1
        
frequencia_genero = spark.createDataFrame(data=freq_genero.items(), schema=['genero', 'frequency'])
frequencia_genero.createOrReplaceTempView('frequencia_genero')
display(frequencia_genero)

In [ ]:
%sql
SELECT * from frequencia_genero
ORDER BY frequency DESC